In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import os 
import PIL
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential 
import time
import keras 
import keras.backend as K 
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose 
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization 
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from IPython.display import SVG 
from keras.utils.vis_utils import model_to_dot
from time import time 
import numpy as np
import pathlib 
from keras_preprocessing.image import ImageDataGenerator
from keras_preprocessing.image import img_to_array
from keras_preprocessing.image import load_img

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dir = '/content/drive/Shareddrives/Graduation Project/Dataset/Teeth images'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) 

train_ds = train_datagen.flow_from_directory(
    dir,
    target_size=(180, 180),
    batch_size=30,
    class_mode='categorical',
    subset='training') # set as training data

val_ds = train_datagen.flow_from_directory(
    dir, # same directory as training data
    target_size=(180, 180),
    batch_size=30,
    class_mode='categorical',
    subset='validation') # set as validation data

In [ ]:
def squeezenet (input_shape, n_classes):

 def fire(m, fs, fe):
  s = Conv2D(fs, 1, activation='relu')(m)
  e1 = Conv2D(fe, 1, activation='relu')(s)
  e3 = Conv2D(fe, 3, padding= 'same', activation='relu')(s)
  output = Concatenate()([e1, e3])
  return output

 input = Input(input_shape)

 m = Conv2D(96, 7, strides=2, padding= 'same', activation='relu')(input)
 m = MaxPool2D(3, strides=2, padding='same')(m)

 m = fire(m, 16, 64)
 m = fire(m, 16, 64)
 m = fire(m, 32, 128)
 m = MaxPool2D (3,strides=2, padding= 'same')(m)

 m = fire(m, 32, 128)
 m = fire(m, 48, 192)
 m = fire(m, 48, 192)
 m = fire(m, 64, 256)
 m = MaxPool2D (3, strides=2, padding='same')(m)

 m = fire(m, 64, 256)
 m = Conv2D(n_classes, 1)(m)
 m = GlobalAvgPool2D()(m)  

 output = Activation('softmax')(m) 
 model = Model(input, output)
 return model

In [ ]:
input_shape=(180, 180,3) 
n_classes =5
K.clear_session()
model = squeezenet(input_shape, n_classes)
model.summary ()
